# Maestría en Ciencia de Datos e Inteligencia Artificial
## Módulo: 09: Minería de Datos
### 2025

### *Msc Renzo Claure*
---

### Algoritmo Apriori

In [ ]:
# pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.2 MB/s eta 0:00:00-:--:--
Note: you may need to restart the kernel to use updated packages.


In [30]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 

In [29]:
#Datos arbitrarios, (solo didactico)
dataset = [
    ['Leche', 'Pan', 'Huevos'],
    ['Leche', 'Pan'],
    ['Pan', 'Huevos'],
    ['Leche', 'Huevos'],
    ['Leche', 'Pan', 'Huevos', 'Cereal'],
    ['Pan', 'Cereal'],
    ['Leche', 'Pan', 'Cereal'],
    ['Leche', 'Pan', 'Huevos', 'Cereal']
]

In [6]:
dataset

[['Leche', 'Pan', 'Huevos'],
 ['Leche', 'Pan'],
 ['Pan', 'Huevos'],
 ['Leche', 'Huevos'],
 ['Leche', 'Pan', 'Huevos', 'Cereal'],
 ['Pan', 'Cereal'],
 ['Leche', 'Pan', 'Cereal'],
 ['Leche', 'Pan', 'Huevos', 'Cereal']]

In [31]:
#OneHotEncoding, Codificar los datos de transacciones
def encode(dataset):
    cleaned_dataset = [[item for item in transaction if not pd.isna(item)] 
        for transaction in dataset]
    encoder = TransactionEncoder()
    encoded_dataset = encoder.fit_transform(cleaned_dataset)
    df = pd.DataFrame(encoded_dataset, columns=encoder.columns_)
    return df

In [32]:
df = encode(dataset)

In [10]:
df

,Cereal,Huevos,Leche,Pan
0,False,True,True,True
1,False,False,True,True
2,False,True,False,True
3,False,True,True,False
4,True,True,True,True
5,True,False,False,True
6,True,False,True,True
7,True,True,True,True


In [33]:
# Aplicar el algoritmo Apriori para encontrar conjuntos de ítems frecuentes
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

In [34]:
frequent_itemsets.sort_values(by='support', ascending=False)

,support,itemsets
3,0.875,(Pan)
2,0.750,(Leche)
1,0.625,(Huevos)
7,0.625,"(Pan, Leche)"
0,0.500,(Cereal)
4,0.500,"(Pan, Cereal)"
5,0.500,"(Huevos, Leche)"
6,0.500,"(Pan, Huevos)"


In [35]:
# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [36]:
print("\nReglas de asociación:")
rules


Reglas de asociación:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Cereal),(Pan),0.500,0.875,0.500,1.000000,1.142857,1.0,0.062500,inf,0.250000,0.571429,1.000000,0.785714
1,(Huevos),(Leche),0.625,0.750,0.500,0.800000,1.066667,1.0,0.031250,1.250,0.166667,0.571429,0.200000,0.733333
2,(Huevos),(Pan),0.625,0.875,0.500,0.800000,0.914286,1.0,-0.046875,0.625,-0.200000,0.500000,-0.600000,0.685714
3,(Pan),(Leche),0.875,0.750,0.625,0.714286,0.952381,1.0,-0.031250,0.875,-0.285714,0.625000,-0.142857,0.773810
4,(Leche),(Pan),0.750,0.875,0.625,0.833333,0.952381,1.0,-0.031250,0.750,-0.166667,0.625000,-0.333333,0.773810


*1. Soporte*  
El soporte de la regla “Leche → Pan” se basa en la fracción de transacciones que contienen ambos items. Aquí, aproximadamente el 62.5% de las transacciones contienen tanto Leche como Pan.  
*2. Confianza*  
De todas las transacciones que contienen Leche, el 83.33% también contienen Pan. Es decir, si en una transacción aparece Leche, hay un 83.33% de probabilidades de que también aparezca Pan.  
*3. Lift*  
Un lift de 0.9524 indica que la ocurrencia conjunta de Leche y Pan es ligeramente inferior a lo que se esperaría si fueran independientes (ya que un lift igual a 1 indicaría independencia). En otras palabras, la presencia de Leche, en este conjunto de transacciones, no aumenta (y de hecho reduce ligeramente) la probabilidad de encontrar Pan.  
*4. Leverage*  
Un leverage de -0.03125 indica que la co-ocurrencia de Leche y Pan es ligeramente menor de lo esperado bajo independencia. El valor negativo significa que, al observar Leche, la aparición conjunta con Pan ocurre con una frecuencia un poco menor de la que se esperaría si ambos ocurrieran de forma independiente.

In [37]:
import numpy as np

In [38]:
def graph_apriori(rules):
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(x) if isinstance(x, frozenset) else x)
    rules['lift_t'] = (rules['lift']-np.min(rules['lift']))/(np.max(rules['lift'])-np.min(rules['lift']))
    fig = px.scatter(rules, x='support', y='confidence', size='lift_t', text='antecedents',
                     hover_data=['support', 'confidence', 'lift'])
    fig.update_traces(textposition='top center')
    fig.update_layout(title='Association Rules', xaxis_title='Support', yaxis_title='Confidence')
    fig.show()    

In [39]:
graph_apriori(rules)

**Análisis Integrado con la Regla "Pan → Leche"**  
Soporte: La cantidad de transacciones con ambos (Pan y Leche) dividida entre el total de transacciones.  
"¿Cuántas veces se ven juntos?"

Confianza: La proporción de transacciones con Pan en las que aparece además Leche.  
"Si veo Pan, ¿cuánto confío en que también esté Leche?"

Lift: Compara la confianza de la regla con la probabilidad general de ver Leche.  
"¿Pan eleva la posibilidad de encontrar Leche?"

Leverage: Mide si Pan y Leche aparecen juntos más (o menos) de lo que esperaríamos por azar.  
"¿Están juntos por encima o por debajo de lo normal?"

Conviction: Evalúa qué tan seguido falla la predicción de Leche cuando se observa Pan.  
"¿Con qué seguridad Pan predice a Leche?"

### Otro ejemplo

In [40]:
dataset = pd.read_csv('files/store_data.csv', header=None)
transactions = []
for i in range(0, 7501):
    transactions.append([str(dataset.values[i,j]) for j in range(0,20)])

In [41]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df = encode(transactions)

In [43]:
encoder = TransactionEncoder()
encoded_dataset = encoder.fit_transform(transactions)
df = pd.DataFrame(encoded_dataset, columns=encoder.columns_)

In [44]:
df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
frequent_itemsets = apriori(df, min_support=0.10, use_colnames=True)

In [46]:
frequent_itemsets

,support,itemsets
0,0.163845,(chocolate)
1,0.179709,(eggs)
2,0.170911,(french fries)
3,0.132116,(green tea)
4,0.129583,(milk)
5,0.238368,(mineral water)
6,0.999867,(nan)
7,0.174110,(spaghetti)
8,0.163845,"(nan, chocolate)"
9,0.179709,"(nan, eggs)"


In [47]:
# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [48]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(chocolate),(nan),0.163845,0.999867,0.163845,1.000000,1.000133,1.0,0.000022,inf,0.000159,0.163867,1.000000,0.581933
1,(eggs),(nan),0.179709,0.999867,0.179709,1.000000,1.000133,1.0,0.000024,inf,0.000163,0.179733,1.000000,0.589867
2,(french fries),(nan),0.170911,0.999867,0.170911,1.000000,1.000133,1.0,0.000023,inf,0.000161,0.170933,1.000000,0.585467
3,(green tea),(nan),0.132116,0.999867,0.131982,0.998991,0.999124,1.0,-0.000116,0.132116,-0.001009,0.131982,-6.569122,0.565495
4,(milk),(nan),0.129583,0.999867,0.129583,1.000000,1.000133,1.0,0.000017,inf,0.000153,0.129600,1.000000,0.564800
5,(mineral water),(nan),0.238368,0.999867,0.238235,0.999441,0.999574,1.0,-0.000102,0.238368,-0.000559,0.238235,-3.195190,0.618854
6,(spaghetti),(nan),0.174110,0.999867,0.174110,1.000000,1.000133,1.0,0.000023,inf,0.000161,0.174133,1.000000,0.587067


In [49]:
graph_apriori(rules)
